# Задание 1. (5 баллов) 
В тетрадке реализована биграмная языковая модель (при генерации учитывается информация только о 1 предыдущем слове). Реализуйте триграмную модель и сгенерируйте несколько текстов. Сравните их с текстами, сгенерированными биграмной моделью. 
Можно использовать те же тексты, что в семинаре, или взять какой-то другой (на английском или русском языке).  

Делать это задание будет легче после прочтения первых 7 страниц вот этой главы из Журафского - https://web.stanford.edu/~jurafsky/slp3/3.pdf


In [0]:
from string import punctuation
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from collections import Counter

In [0]:
!wget https://raw.githubusercontent.com/SvA1/texts/master/dostoevsky.txt
dostoevsky = open('dostoevsky.txt').read()

In [0]:
def normalize(text):
    normalized_text = [word.strip(punctuation) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text


In [0]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [0]:
sentences_dostoevsky = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]

In [0]:
unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()
trigrams_dostoevsky = Counter()

for sentence in sentences_dostoevsky:
    unigrams_dostoevsky.update(sentence)
    bigrams_dostoevsky.update(ngrammer(sentence))
    trigrams_dostoevsky.update(ngrammer(sentence, n=3))

In [0]:
id2word_unigrams = list(unigrams_dostoevsky)
word2id_unigrams = {word:i for i, word in enumerate(id2word_unigrams)}

id2word_bigrams = list(bigrams_dostoevsky)
word2id_bigrams = {word:i for i, word in enumerate(id2word_bigrams)}

In [0]:
matrix_dostoevsky = np.zeros((len(bigrams_dostoevsky), 
                   len(unigrams_dostoevsky)))

for ngram in trigrams_dostoevsky:
    word1, word2, word3 = ngram.split()
    bigram = '{} {}'.format(word1, word2)
    matrix_dostoevsky[word2id_bigrams[bigram]][word2id_unigrams[word3]] =  (trigrams_dostoevsky[ngram]/
                                                                                 bigrams_dostoevsky[bigram])

In [0]:
matrix_dostoevsky_bigrams = np.zeros((len(unigrams_dostoevsky), 
                   len(unigrams_dostoevsky)))

for ngram in bigrams_dostoevsky:
    word1, word2 = ngram.split()
    matrix_dostoevsky_bigrams[word2id_unigrams[word1]][word2id_unigrams[word2]] =  (bigrams_dostoevsky[ngram]/
                                                                     unigrams_dostoevsky[word1])

### Текст, сгенерированный биграммами

In [10]:

def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)
  
print(generate(matrix_dostoevsky_bigrams, id2word_unigrams, word2id_unigrams).replace('<end>', '\n'))

опять окликает астафий иваныч 
 э-эх 
 думаю воротится к тому попал я их взял я осерчал возьмет астафий иванович пустился вслед за него под кровать на коленках передо мной в садовой 
 я сына родного хороню 
 посидел я видел что же смекаю про себя посмотрел на приступочку голову срезал бы долго лежал он сударь так 
 ну сам знаю зачем там и домоводка и не сводит а он сделает как вот говорю их помещик который был говорю тебя совсем собрался идти искать положил на меня обокрал да еще с утра видел что какой-то пожилой человек уговорил или что-нибудь


### Текст, сгенерированный триграммами

In [0]:
def start_step(text, matrix_2, word2id_bigrams, id2word, start_idx):
    second_word = np.random.choice(matrix_2.shape[1], p=matrix_2[start_idx])
    word = id2word[second_word]
    text.append(word)
    current_bigram = "<start> {}".format(word)
    current_idx = word2id_bigrams[current_bigram]
    return current_bigram, current_idx

def generate(matrix, matrix_2, id2word, word2id_bigrams, word2id_unigrams, n=100):
    text = []
    start_idx = word2id_unigrams["<start>"]
    current_bigram, current_idx = start_step(text, matrix_2, word2id_bigrams, id2word, start_idx)
    for i in range(n):

        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        word = id2word[chosen]
        
        if word == '<end>':
            current_bigram, current_idx = start_step(text, matrix_2, word2id_bigrams, id2word, start_idx)
        else:
            text.append(word)
            current_bigram = "{} {}".format(current_bigram.split()[1], word)
            current_idx = word2id_bigrams[current_bigram]
    
    return ' '.join(text)

In [12]:

print(generate(matrix_dostoevsky, matrix_dostoevsky_bigrams, id2word_unigrams, word2id_bigrams, word2id_unigrams, n=100).replace('<end>', '\n'))

жилетку чиновнику что под нами жил переделывал думаю емеля емеля оно вот как пьянство человека испортит что ж он такой как на честного вора наутро только смотрю лежит себе на голом полу скрючившись в своей шинелишке унизился больно так и весь вечер просидел не шелохнулся уж я признаться сказать перетрусил да и улизнет поминай как звали я уж сегодня обедать не буду больше пить астафий иваныч я лучше уж пойду-с то есть энтого их не брал-с да хоть бы ты портняжному-то искусству повыучился да так нездоровится астафий иваныч уж лучше сгори вещь а вору уступить досадно не хочется я как выслушал


И биграммная и триграммная языковые модели сгенерировали довольно осмысленные тексты. 
Тем не менее,есть места, где в триграммной модели текст более связный, особенно если сравнить длинное предложение, сгенерированной биграммной моделью ('и домоводка и не сводит а он сделает') и триграммной. В триграммной модели однако есть и такие места, где теряется связность текста, например, 'не буду больше пить астафий иваныч я лучше уж пойду-с то есть энтого'.
В целом, на мой взгляд, биграммная модель сгенерировала текст не сильно отличающийся по осмысленности от триграммной. Возможно это из-за того, что текст, на котором генерируются эти модели, довольно маленький по объему.

# Задание 2. (5 баллов) 
При помощи gensim.models.Phrases реализуйте byte-pair-encoding, про который говорилось на первом семинаре (https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/Preprocessing.ipynb) 
А именно 1) возьмите любой текст; разбейте его на предложения, а каждое предложение разбейте на отдельные символы (не потеряйте пробелы) 2) обучите gensim.models.Phrases на полученных символьных предложениях 3) примените полученный нграммер к этим символьным предложениям 4) повторите 2 и 3 N количество раз, чтобы начали получаться целые слова
Параметры в gensim.models.Phrases влияют на количество получаемых нграммов после каждого прохода, поэтому не забудьте их настроить

In [0]:
import re, collections
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import Text8Corpus
from gensim.test.utils import datapath

In [0]:
text = '''Национальный центр кибербезопасности Великобритании (NCSC) опубликовал результаты исследования, проведенного совместно с Агентством национальной безопасности США. Доклад посвящен известной русскоязычной хакерской группировке Turla и ее новому методу работы — она проникла в инфраструктуру иранских хакеров, «украла» у иранцев доступ к взломанным ими серверам и сам хакерский инструментарий, который теперь использует в собственных целях.

«Группа Turla, также известная как „Водяной жук“ и „Ядовитый медведь“, широко известна как группа, связанная с силами из России», — говорится в предисловии к докладу. Она занимается взломом правительственных, военных, энергетических и других организаций с целью сбора данных. На Западе ее связывают с российскими спецслужбами. Ранее NCSC обнаружила, что хакеры из этой группировки используют программные «закладки» под названием Neuron и Nautilus, которые позволяют взламывать серверы на платформах Microsoft Windows. После дополнительного анализа эксперты пришли к выводу, что Neuron и Nautilus — это софт иранских хакеров. Причем сами иранцы, которые их создали и использовали, «почти наверняка не знали об использовании Turla своих программных закладок и не были замешаны в их использовании», полагают в NCSC.

Сначала россияне находили серверы, уже взломанные иранцами, и брали их под собственный контроль, а потом развернули чужой хакерский инструментарий против новых жертв. «Turla сканировала серверы на наличие иранских „закладок“ и пыталась использовать их для укрепления своих позиций. Основное их внимание при этом было сосредоточено на Ближнем Востоке», — говорится в докладе.

Спецслужбы полагают, что Turla смогла проделать эту работу, взломав самих иранских коллег. «Чтобы установить связь с „закладками“, Turla должна была иметь доступ к соответствующим криптографическим ключам и, вероятно, доступ к контролирующему их программному обеспечению», — поясняют NCSC.

Более того, группировка использовала в качестве инструмента иранскую сетевую инфраструктуру. На иранских серверах Turla установила собственные закладки — и в результате получила доступ к файлам иранских хакеров, а также «беспрецедентную осведомленность о тактике, методах и процедурах иранских APT, включая списки жертв и учетные данные для доступа к их инфраструктуре — а также код, необходимый для создания новых версий таких инструментов, как Neuron».'''

In [0]:
sentences = sent_tokenize(text)

In [0]:
sentences

['Национальный центр кибербезопасности Великобритании (NCSC) опубликовал результаты исследования, проведенного совместно с Агентством национальной безопасности США.',
 'Доклад посвящен известной русскоязычной хакерской группировке Turla и ее новому методу работы — она проникла в инфраструктуру иранских хакеров, «украла» у иранцев доступ к взломанным ими серверам и сам хакерский инструментарий, который теперь использует в собственных целях.',
 '«Группа Turla, также известная как „Водяной жук“ и „Ядовитый медведь“, широко известна как группа, связанная с силами из России», — говорится в предисловии к докладу.',
 'Она занимается взломом правительственных, военных, энергетических и других организаций с целью сбора данных.',
 'На Западе ее связывают с российскими спецслужбами.',
 'Ранее NCSC обнаружила, что хакеры из этой группировки используют программные «закладки» под названием Neuron и Nautilus, которые позволяют взламывать серверы на платформах Microsoft Windows.',
 'После дополнительн

In [0]:
def split(word): 
    return [char for char in word]  

In [0]:

vocab = []
for i in range(len(sentences)):
    vocab.append(split(sentences[i]))

In [0]:
print(vocab)

[['Н', 'а', 'ц', 'и', 'о', 'н', 'а', 'л', 'ь', 'н', 'ы', 'й', ' ', 'ц', 'е', 'н', 'т', 'р', ' ', 'к', 'и', 'б', 'е', 'р', 'б', 'е', 'з', 'о', 'п', 'а', 'с', 'н', 'о', 'с', 'т', 'и', ' ', 'В', 'е', 'л', 'и', 'к', 'о', 'б', 'р', 'и', 'т', 'а', 'н', 'и', 'и', ' ', '(', 'N', 'C', 'S', 'C', ')', ' ', 'о', 'п', 'у', 'б', 'л', 'и', 'к', 'о', 'в', 'а', 'л', ' ', 'р', 'е', 'з', 'у', 'л', 'ь', 'т', 'а', 'т', 'ы', ' ', 'и', 'с', 'с', 'л', 'е', 'д', 'о', 'в', 'а', 'н', 'и', 'я', ',', ' ', 'п', 'р', 'о', 'в', 'е', 'д', 'е', 'н', 'н', 'о', 'г', 'о', ' ', 'с', 'о', 'в', 'м', 'е', 'с', 'т', 'н', 'о', ' ', 'с', ' ', 'А', 'г', 'е', 'н', 'т', 'с', 'т', 'в', 'о', 'м', ' ', 'н', 'а', 'ц', 'и', 'о', 'н', 'а', 'л', 'ь', 'н', 'о', 'й', ' ', 'б', 'е', 'з', 'о', 'п', 'а', 'с', 'н', 'о', 'с', 'т', 'и', ' ', 'С', 'Ш', 'А', '.'], ['Д', 'о', 'к', 'л', 'а', 'д', ' ', 'п', 'о', 'с', 'в', 'я', 'щ', 'е', 'н', ' ', 'и', 'з', 'в', 'е', 'с', 'т', 'н', 'о', 'й', ' ', 'р', 'у', 'с', 'с', 'к', 'о', 'я', 'з', 'ы', 'ч', 'н', '

In [0]:
phrases = Phrases(vocab, min_count=19, scoring='npmi', threshold=-1)

In [0]:
p0=Phrases (phrases[vocab], min_count=10, scoring='npmi', threshold=-1)

In [0]:
p1=Phrases (p0[phrases[vocab]], min_count=1, scoring='npmi', threshold=0)

In [0]:
p2=Phrases (p1[p0[phrases[vocab]]], min_count=2, scoring='npmi', threshold=0.1)

In [0]:
print(list(p2[p1[p0[vocab]]]))

[['Н_а', 'ц_и_о_н_а', 'л_ь_н_ы', 'й_ _ц', 'е_н_т', 'р_ _к', 'и', 'б_е_р', 'б_е', 'з_о', 'п_а', 'с_н_о', 'с_т_и_ ', 'В_е', 'л_и', 'к_о', 'б_р', 'и_т', 'а_н_и', 'и_ _(', 'N_C', 'S_C', ')_ ', 'о_п', 'у_б', 'л_и', 'к_о_в', 'а_л', ' _р', 'е_з', 'у_л_ь', 'т_а', 'т_ы', ' _и_с', 'с_л', 'е_д_о', 'в', 'а_н_и', 'я', ',', ' _п_р_о', 'в_е_д', 'е_н_н_о', 'г_о', ' _с_о_в', 'м_е_с_т', 'н_о_ _с', ' _А', 'г_е_н', 'т_с_т', 'в_о_м', ' _н_а', 'ц_и_о_н_а', 'л_ь_н_о', 'й_ _б', 'е_з', 'о_п', 'а_с', 'н_о_с_т', 'и_ _С', 'Ш_А', '.'], ['Д_о', 'к_л_а', 'д_ _п', 'о_с', 'в_я', 'щ_е_н', ' _и_з_в_е_с_т', 'н_о_й_ ', 'р_у_с', 'с_к_о', 'я_з', 'ы_ч', 'н_о_й_ ', 'х_а_к', 'е_р_с_к', 'о_й_ ', 'г_р_у_п_п', 'и_р_о_в_к', 'е_ _T', 'u_r_l', 'a_ _и', ' ', 'е_е_ ', 'н_о_в', 'о_м_у', ' _м_е', 'т_о_д', 'у_ ', 'р_а_б', 'о_т', 'ы_ ', '—_ ', 'о_н_а', ' _п_р_о', 'н_и', 'к_л_а', ' _в_ _и', 'н_ф_р_а_с_т', 'р_у_к_т_у', 'р_у_ _и', 'р_а_н', 'с_к_и_х_ _х', 'а_к_е_р', 'о_в', ',', ' ', '«_у', 'к_р_а', 'л_а_»', ' _у', ' _и', 'р_а_н', 'ц_е', 'в_ _

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
phrases_input = phrases[vocab]
for i in range(100):
    phrases = Phrases(phrases_input, min_count=1, scoring='npmi', threshold=-1)
    phrases_input = phrases[phrases_input]

In [0]:
print(list(phrases[phrases_input]))

[['Н_а_ц_и_о_н_а_л_ь_н_ы_й_ _ц_е_н_т_р_ _к_и_б_е_р_б_е_з_о_п_а_с_н_о_с_т_и_ _В_е_л_и_к_о_б_р_и_т_а_н_и_и_ _(_N_C_S_C_)_ _о_п_у_б_л_и_к_о_в_а_л_ _р_е_з_у_л_ь_т_а_т_ы_ _и_с_с_л_е_д_о_в_а_н_и_я_,_ _п_р_о_в_е_д_е_н_н_о_г_о_ _с_о_в_м_е_с_т_н_о_ _с_ _А_г_е_н_т_с_т_в_о_м_ _н_а_ц_и_о_н_а_л_ь_н_о_й_ _б_е_з_о_п_а_с_н_о_с_т_и_ _С_Ш_А_.'], ['Д_о_к_л_а_д_ _п_о_с_в_я_щ_е_н_ _и_з_в_е_с_т_н_о_й_ _р_у_с_с_к_о_я_з_ы_ч_н_о_й_ _х_а_к_е_р_с_к_о_й_ _г_р_у_п_п_и_р_о_в_к_е_ _T_u_r_l_a_ _и_ _е_е_ _н_о_в_о_м_у_ _м_е_т_о_д_у_ _р_а_б_о_т_ы_ _—_ _о_н_а_ _п_р_о_н_и_к_л_а_ _в_ _и_н_ф_р_а_с_т_р_у_к_т_у_р_у_ _и_р_а_н_с_к_и_х_ _х_а_к_е_р_о_в_,_ _«_у_к_р_а_л_а_»_ _у_ _и_р_а_н_ц_е_в_ _д_о_с_т_у_п_ _к_ _в_з_л_о_м_а_н_н_ы_м_ _и_м_и_ _с_е_р_в_е_р_а_м_ _и_ _с_а_м_ _х_а_к_е_р_с_к_и_й_ _и_н_с_т_р_у_м_е_н_т_а_р_и_й_,_ _к_о_т_о_р_ы_й_ _т_е_п_е_р_ь_ _и_с_п_о_л_ь_з_у_е_т_ _в_ _с_о_б_с_т_в_е_н_н_ы_х_ _ц_е_л_я_х_.'], ['«_Г_р_у_п_п_а_ _T_u_r_l_a_,_ _т_а_к_ж_е_ _и_з_в_е_с_т_н_а_я_ _к_а_к_ _„_В_о_д_я_н_о_й_ _ж_у_к_“_ _и_ _„_Я_д_о_в_и_т

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
